# Rett Video Circle Features

I'm trying to distinguish the good from the bad

In [ ]:
# experiment with auto reloading
%load_ext autoreload
%autoreload 1

In [ ]:
import pickle
positive = pickle.load(open('MSB_Video_1_positive.blobs', 'rb'))
negative = pickle.load(open('MSB_Video_1_negative.blobs', 'rb'))
both = positive.copy()
both.extend(negative)

In [ ]:
%aimport ImageChooser
from ImageChooser import ImageChooser

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 200

def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])
        
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)

def circularity(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    area = cv2.contourArea(contour)
    result = 4 * np.pi * (area / perimeter ** 2)
    return result

def isCircular(contour, hull=False):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    if hull:
        contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    return 0.7 <= circularity <= 1.2
    # return 0.5 <= circularity <= 1.4

def isInside(contour, size):
    r = np.zeros(size[:2], dtype=np.uint8)
    cv2.drawContours(r, [ contour - np.min(contour, axis=0) ], -1, 255, cv2.FILLED)
    return r == 255

In [ ]:
show(isInside(positive[0][1], positive[0][2].shape))

In [ ]:
def getFeatures(blobs, value):
    features = []
    for fno, contour, pixels in blobs:
        bim = isBlue(pixels)
        h, w = bim.shape
        circ = circularity(contour)
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        roughness = cv2.arcLength(cv2.convexHull(contour), True) / perimeter
        isin = isInside(contour, bim.shape)
        fracin = np.sum(isin) / (h*w)
        std0 = np.std(bim[isin])
        std1 = np.std(cv2.dilate(bim, np.ones((3,3)))[isin])
        features.append([circ, perimeter, area, roughness, fracin, std0, std1])
    features = np.array(features)
    values = value * np.ones(len(blobs))
    return features, values

In [ ]:
pos_data, pos_value = getFeatures(positive, 1)
neg_data, neg_value = getFeatures(negative, 0)
data = np.concatenate([pos_data, neg_data])
value = np.concatenate([pos_value, neg_value])

In [ ]:
value[918] = 1
value[917] = 1
value[1259] = 1
value[1260] = 1

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

model = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, value, test_size=0.1)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
model.named_steps['logisticregression'].n_iter_

In [ ]:
model.named_steps['logisticregression'].coef_

In [ ]:
model.named_steps['logisticregression'].intercept_

In [ ]:
score = model.score(x_test, y_test)
print(score)

In [ ]:
model.score(data, value)

In [ ]:
pickle.dump(model, open('models/LR1.pkl', 'wb'))

In [ ]:
for i, p in enumerate(logisticRegr.predict(data_std)):
    if p != value[i]:
        show(both[i][2], title="{} should be {}".format(i, value[i]))

In [ ]:
goodBlobs = [ both[i] for i, p in enumerate(logisticRegr.predict(data_std)) if p ]
goodBlobs.sort(key = lambda b: b[0])

In [ ]:
def plotBlobs(blobs):
    ncols = 10
    nrows = (len(blobs) + ncols - 1) // ncols
    perCol=1.7
    plt.figure(figsize=(perCol*ncols,perCol*nrows), dpi=100)
    for i, (fno, blob, pixels) in enumerate(blobs):
        plt.subplot(nrows, ncols, i+1)
        im = cv2.cvtColor(pixels, cv2.COLOR_LAB2RGB)
        plt.imshow(im)
        plt.axis('off')

## Good

In [ ]:
plotBlobs(goodBlobs)

## Fool with dct

In [ ]:
im = positive[0][2]
show(im)
bim = isBlue(im)
show(bim)

In [ ]:
from scipy.fftpack import dct
def dct2(im):
    return dct(dct(im, axis=0, norm='ortho'), axis=1, norm='ortho')


def blockDCT2(im):
    ishape = np.array(im.shape)
    oshape = ((ishape + 7) // 8) * 8
    pads = oshape - ishape
    im = np.pad(im, pads, 'symmetric')
    
    s = np.zeros((8,8))
    d = 0
    for r in range(0, oshape[0], 8):
        for c in range(0, oshape[1], 8):
            b = im[r:r+8,c:c+8]
            s += dct2(b - np.mean(b))**2
            d += 1
    return np.sqrt(s / d)


def meanBlockDCT2(im, axis=0):
    '''block dct of mean of given axis'''
    m = np.mean(im, axis=axis)
    r = np.zeros(8 * ((len(m) + 7) // 8))
    r[0:len(m)] = m
    
    s = np.zeros(8)
    d = 0
    for i in range(0, len(r)-4, 4):
        b = r[i:i+8]
        s += dct(b - np.mean(b), norm='ortho')**2
        d += 1
    return np.sqrt(s / d)


def idct2(im):
    return dct(dct(im, axis=0, norm='ortho', type=3), axis=1, norm='ortho', type=3)


def look(im):
    bim = isBlue(im)
    plt.plot(meanBlockDCT2(bim))
    plt.plot(meanBlockDCT2(bim, axis=1))
    show(blockDCT2(bim))
    show(bim)
look(im)

In [ ]:
look(negative[2][2])

In [ ]:
look(positive[2][2])

In [ ]:
s = [ blob for blob in positive if blob[0] == 1717 ]
look(s[0][2])

In [ ]:
show(idct2(dct2(im)))

In [ ]:
t = np.zeros((16,16))
t[0,2] = 1
show(idct2(t))

In [ ]:
i = np.zeros((16,16))
i[:,::3] = 1
show(i)
show(dct2(i))

## Fool with those lines that distore the contour

In [ ]:
bad = both[626]
show(bad[2])
bbad = isBlue(bad)



In [ ]:
def findBlobs(image, verbose=True):
    # find the special blue
    blue = isBlue(image)
    _, blue = cv2.threshold(blue, 0.6, 255, cv2.THRESH_BINARY)
    blue = blue.astype(np.uint8)
    # dilate a bit to fill in noise
    blue = cv2.dilate(blue, np.ones((3,3), dtype=np.uint8), iterations=1)
    blue = cv2.erode(blue, np.ones((3,3), dtype=np.uint8), iterations=1)
    
    if verbose:
        show(blue)
    # get the contours of the blue regions
    im2, contours, hierarchy = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if verbose:
        oim = image.copy()
        cv2.drawContours(oim, contours, -1, (0,0,0), 1)
        show(oim)
    # filter by area
    if True:
        minArea = np.pi * 4**2
        contours = [np.squeeze(contour) for contour in contours if minArea < cv2.contourArea(contour)]
        if verbose:
            oim = image.copy()
            cv2.drawContours(oim, contours, -1, (0,0,0), 1)
            show(oim)
    if False:
        # filter by circularity
        contours = [contour for contour in contours if isCircular(contour)]
        if verbose:
            oim = image.copy()
            cv2.drawContours(oim, contours, -1, (0,0,0), 1)
            show(oim)

    return contours

In [ ]:
contours = findBlobs(bad[2])

In [ ]:
epsilon = 0.03*cv2.arcLength(c[0],True)
approx = cv2.approxPolyDP(c[0],epsilon,True)
oim = bad[2].copy()
cv2.drawContours(oim, approx, -1, (0,0,0), 1)
show(oim)


In [ ]:
c = contours[0]

In [ ]:
a = cv2.contourArea(c)
print(a)
cm = np.mean(c, axis=0)
d = np.sum((c - cm)**2, axis=1)
c = c[d != np.max(d)]
oim = bad[2].copy()
cv2.drawContours(oim, [c], -1, (0,0,0), 1)
show(oim)